# **Estimation of freshwater lens thickness (_FWL_) using three different methods**

> **Objective:** Apply three different methods to estimate the depth of the freshwater lens. Based on this estimation, extract the corresponding array for the freshwater zone, calculate basic statistics, and plot the boxplots for each profile using the three methods.

---

### Import Libraries

In [1]:
import sys
import os
import glob


root = os.path.abspath('../..')  
sys.path.append(root)

import pandas as pd
import numpy as np

from typing import Tuple, Optional, Any, Dict

import piecewise_regression
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import modules.statistics_fwl_estimation as st_fwl

from modules import processing, load, plots, analysis

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 25)

---

### **Method 1:** _Intuitive criterion_ **(IC)** y **Method 2:** _Optimal BIC_ **(BIC)**

1. Load data `selection_fwl_ic_bic_dgh.csv`

In [2]:
df_estimations = pd.read_csv(f'{root}/data/selection_fwl_ic_bic_dgh.csv')
try:
    df_estimations.loc[df_estimations['profile'] == 'LRS65D_YSI_20230827', 'profile'] = 'LRS65D_YSI_20220812'
    df_estimations.rename(columns={'profile': 'ID'}, inplace=True)
except KeyError:
    df_estimations.loc[df_estimations['ID'] == 'LRS65D_YSI_20230827', 'ID'] = 'LRS65D_YSI_20220812'

df_estimations


,ID,total_bp_ic,fwl_bp_ic,vp_fwl_ic,total_bp_bic,fwl_bp_bic,vp_fwl_bic,vp_fwl_dgh,COMENTARIOS,SEC Profile Features,vp_fwl_lrst_x,vp_fwl_lrst_y
0,AW1D_YSI_20230826,6,2,16.317073,3,1,17.169116,16.386594,NaN,NaN,6.120,6.120
1,AW2D_YSI_20230815,7,2,9.563224,3,1,9.907636,11.319882,NaN,NaN,13.103,13.103
2,AW5D_YSI_20230824,5,1,14.134225,4,1,13.652858,12.589484,NaN,NaN,12.580,12.580
3,AW6D_YSI_20230815,5,2,13.216555,2,1,13.561559,13.844801,NaN,NaN,13.016,13.016
4,AW7D_YSI_20230814,2,1,13.290227,2,1,13.290227,13.316418,No está correctamente cortado el perfil,NaN,12.881,12.881
5,BW1D_YSI_20230824,3,1,14.125652,3,1,14.125652,14.117028,NaN,NaN,NaN,NaN
6,BW2D_YSI_20230819,3,1,13.997452,3,1,13.997452,14.581115,NaN,NaN,14.038,14.038
7,BW3D_YSI_20230818,4,1,11.701564,4,1,11.701564,12.974280,NaN,NaN,10.784,10.784
8,BW4D_YSI_20230816,2,1,14.358411,2,1,14.358411,14.124974,NaN,NaN,14.014,14.014
9,BW5D_YSI_20230822,4,2,12.393402,2,1,11.527182,10.262300,NaN,NaN,NaN,NaN


---

### **Method 3:** _Dupuit-Ghyben-Herzberg_ **(DGH)**

1. Functions to implement DGH

In [3]:
def extract_single_breakpoint(model: Any) -> Tuple[Optional[float], Optional[float],
                                                    Optional[Tuple[float, float]],
                                                    Optional[Tuple[float, float]]]:
    """
    Extracts the estimated breakpoint (in sec) and its predicted vertical position (vp) along with 
    their confidence intervals from a fitted piecewise regression model.

    The function assumes that the model contains one breakpoint and that the underlying 
    piecewise_regression model provides an attribute 'best_muggeo' with the following structure:
        - best_muggeo.best_fit.estimates: a dict with keys "breakpoint1" containing:
            - "estimate": the breakpoint value (sec)
            - "confidence_interval": a tuple (lower_bound, upper_bound) for the breakpoint estimate.
        - model.predict(...): returns the predicted vp given a sec value.

    For the vp confidence interval, the function predicts vp at both endpoints of the sec confidence interval.

    Parameters:
        model (Any): Fitted piecewise regression model (instance of piecewise_regression.main.Fit).

    Returns:
        Tuple containing:
            - bp_sec (float): Estimated breakpoint in sec.
            - bp_vp (float): Estimated vertical position at the breakpoint.
            - conf_interval_sec (tuple): Confidence interval (lower, upper) for the sec breakpoint.
            - conf_interval_vp (tuple): Confidence interval (lower, upper) for the predicted vp.
        If extraction fails, all values will be None.
    """
    try:
        if not hasattr(model, 'best_muggeo') or not model.best_muggeo:
            raise ValueError("The model did not converge or lacks valid breakpoints.")

        estimates = model.best_muggeo.best_fit.estimates
        bp_key = "breakpoint1"
        bp_sec = estimates[bp_key]["estimate"]
        conf_interval_sec = estimates[bp_key]["confidence_interval"]  # (lower, upper)
        bp_vp = model.predict(np.array([bp_sec]))[0]
        
        # Calculate vp confidence interval by predicting at the sec confidence interval boundaries
        conf_interval_vp = (
            model.predict(np.array([conf_interval_sec[0]]))[0],
            model.predict(np.array([conf_interval_sec[1]]))[0]
        )
        return bp_sec, bp_vp, conf_interval_sec, conf_interval_vp
    except Exception as e:
        # In a production environment you may want to log the exception details.
        return None, None, None, None

def dgh_fwl_estimation(path: str, 
                      vp_column: str, 
                      sec_column: str, 
                      threshold: float = 27800, # uS/cm  
                      tolerance: float = 1e-5, 
                      min_distance: float = 0.01
                     ) -> Tuple[pd.DataFrame, Dict[str, Dict[str, Any]]]:
    """
    Processes well data CSV files, splits each file's data based on a conductivity threshold (DGH method), 
    applies piecewise linear regression (with 1 breakpoint) to each segment, and returns both 
    a summary DataFrame with the estimated breakpoint positions and their confidence intervals, 
    as well as a dictionary containing the fitted models for each well.

    For each CSV file in the provided directory, the function:
      1. Reads the data.
      2. Filters the specified columns for vertical position (vp) and conductivity (sec).
      3. Splits the data into two segments:
            - Segment 1: where sec <= threshold
            - Segment 2: where sec > threshold
      4. Runs a piecewise regression on each segment using:
            piecewise_regression.Fit(sec, vp, n_breakpoints=1)
      5. Extracts the breakpoint information using the helper function `extract_single_breakpoint`.
      6. Builds a summary DataFrame and a dictionary mapping each well's ID to its fitted models.

    Parameters:
        path (str): Directory path containing the well data CSV files.
        vp_column (str): Name of the column containing vertical position values.
        sec_column (str): Name of the column containing electrical conductivity values.
        threshold (float): Conductivity threshold (µS/cm) to split the data. Default is 27800.
        tolerance (float): Tolerance for convergence in the piecewise regression. Default is 1e-5.
        min_distance (float): Minimum distance between breakpoints. Default is 0.01.

    Returns:
        Tuple containing:
            - pd.DataFrame: Summary DataFrame with breakpoint estimates and confidence intervals for each well.
            - dict: Dictionary with keys as well IDs and values as dictionaries containing the fitted models 
                    for the low and high conductivity segments, e.g.:
                        {
                            "well_id": {
                                "low": <model_low_object or None>,
                                "high": <model_high_object or None>
                            },
                            ...
                        }
    """
    results = []
    models: Dict[str, Dict[str, Any]] = {}
    
    # Validate if the provided path exists
    if not os.path.exists(path):
        raise FileNotFoundError(f"The provided path '{path}' does not exist.")
    
    # Find all CSV files in the directory
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    if not csv_files:
        raise ValueError(f"No CSV files found in the directory '{path}'.")
    
    for file in csv_files:
        try:
            df = pd.read_csv(file)
        except Exception:
            # Skip file if it cannot be read
            continue
        
        # Check if the required columns are present
        if vp_column not in df.columns or sec_column not in df.columns:
            continue
        
        # Filter and drop missing values for the required columns
        df_filtered = df[[vp_column, sec_column]].dropna()
        
        # Split the data based on the conductivity threshold
        df_low = df_filtered[df_filtered[sec_column] <= threshold]
        df_high = df_filtered[df_filtered[sec_column] > threshold]
        
        # Initialize results for each segment and store models for this well
        bp_sec_low, bp_vp_low, conf_sec_low, conf_vp_low = (None, None, None, None)
        bp_sec_high, bp_vp_high, conf_sec_high, conf_vp_high = (None, None, None, None)
        well_models = {"low": None, "high": None}
        
        # Process low conductivity segment if sufficient data exists
        if not df_low.empty and len(df_low) > 1:
            try:
                # Run piecewise regression: sec as independent variable, vp as dependent variable
                model_low = piecewise_regression.Fit(
                    df_low[sec_column].values,
                    df_low[vp_column].values,
                    n_breakpoints=1,
                    tolerance=tolerance,
                    min_distance_between_breakpoints=min_distance
                )
                well_models["low"] = model_low
                bp_sec_low, bp_vp_low, conf_sec_low, conf_vp_low = extract_single_breakpoint(model_low)
            except Exception:
                well_models["low"] = None
                bp_sec_low, bp_vp_low, conf_sec_low, conf_vp_low = (None, None, None, None)
        
        # Process high conductivity segment if sufficient data exists
        if not df_high.empty and len(df_high) > 1:
            try:
                model_high = piecewise_regression.Fit(
                    df_high[sec_column].values,
                    df_high[vp_column].values,
                    n_breakpoints=1
                )
                well_models["high"] = model_high
                bp_sec_high, bp_vp_high, conf_sec_high, conf_vp_high = extract_single_breakpoint(model_high)
            except Exception:
                well_models["high"] = None
                bp_sec_high, bp_vp_high, conf_sec_high, conf_vp_high = (None, None, None, None)
        
        # Extract well ID from the filename (e.g., base name without extension)
        well_id = os.path.splitext(os.path.basename(file))[0]
        models[well_id] = well_models
        
        # Append the results for the current well
        results.append({
            "ID": well_id,
            "breakpoint_1 (sec)": bp_sec_low,
            "breakpoint_2 (sec)": bp_sec_high,
            "breakpoint_1 (vp)": bp_vp_low,
            "breakpoint_2 (vp)": bp_vp_high,
            "confidence_interval_1 (sec)": conf_sec_low,
            "confidence_interval_2 (sec)": conf_sec_high,
            "confidence_interval_1 (vp)": conf_vp_low,
            "confidence_interval_2 (vp)": conf_vp_high,
        })
    
    summary_df = pd.DataFrame(results)
    return summary_df, models

2. Extract the depths of freshwater lenses for `rawdy` wells.

In [4]:
dgh_df, models = dgh_fwl_estimation(path = f'{root}/data/rawdy', # Change to the desired path
                  vp_column = 'Vertical Position [m]', 
                  sec_column = 'Corrected sp Cond [uS/cm]', 
                  threshold = 27800, # uS/cm
                  tolerance = 1e-5,
                  min_distance = 0.01)

c:\Users\Arhui\Desktop\proyectos\mar\freshwater_lens\.venv\Lib\site-packages\piecewise_regression\davies.py:25: RuntimeWarning: invalid value encountered in scalar divide
  return beta_hat/se_beta_hat
c:\Users\Arhui\Desktop\proyectos\mar\freshwater_lens\.venv\Lib\site-packages\piecewise_regression\davies.py:25: RuntimeWarning: invalid value encountered in scalar divide
  return beta_hat/se_beta_hat


In [5]:
dgh_df["ID"] = dgh_df["ID"].str.replace("_rowdy", "", regex=False)

dgh_df

,ID,breakpoint_1 (sec),breakpoint_2 (sec),breakpoint_1 (vp),breakpoint_2 (vp),confidence_interval_1 (sec),confidence_interval_2 (sec),confidence_interval_1 (vp),confidence_interval_2 (vp)
0,AW1D_YSI_20230826,1782.749922,37909.140859,16.386594,21.442094,"(1772.7607620488282, 1792.7390811216121)","(37809.66181192774, 38008.61990587464)","(16.045029226878654, 16.387895260811653)","(21.427465557974756, 21.476743814381553)"
1,AW2D_YSI_20230815,486.851662,NaN,11.319882,NaN,"(485.56222481190713, 488.1410999704472)",None,"(11.183451553508576, 11.320780195880054)",None
2,AW5D_YSI_20230824,834.756079,36661.081674,12.090368,23.999913,"(831.2568398659375, 838.2553190509141)","(36486.77331297517, 36835.390034960816)","(11.875542297176125, 12.092036202265469)","(23.96587690287194, 24.010105194348697)"
3,AW6D_YSI_20230815,954.002106,29720.766275,13.844801,21.420759,"(922.358286338224, 985.6459264912054)","(29489.762397085542, 29951.770152588295)","(13.334872626476601, 13.854883063137986)","(21.365516546564884, 21.439678293634405)"
4,AW7D_YSI_20230814,1087.127912,44495.619235,13.316418,25.752851,"(1079.5881685069842, 1094.6676545994799)","(44383.077194992096, 44608.161275187864)","(13.20728356916882, 13.319302635184874)","(25.732613380920004, 25.791591979180666)"
5,BW10D_YSI_20230825,1311.614790,51914.807199,11.276226,21.648532,"(1310.9608670550015, 1312.268713121443)","(51900.49793991486, 51929.11645894492)","(11.051473324776282, 11.276614215499183)","(21.645965121927496, 21.8068827343101)"
6,BW11D_YSI_20230823,523.124654,48862.791632,11.645055,16.667353,"(521.5659236545443, 524.6833849058869)","(48790.21340306066, 48935.36986144525)","(11.41482809953466, 11.645406136001807)","(16.661741223347526, 16.78651657672682)"
7,BW1D_YSI_20230824,3519.188552,51850.375636,14.117028,22.376200,"(3493.690023211311, 3544.6870811676063)","(51835.854230666184, 51864.89704060214)","(13.733089306231207, 14.120270377697041)","(22.372605411748086, 22.431709971717147)"
8,BW2D_YSI_20230819,1334.805145,32491.136128,14.581115,24.051731,"(1324.0861414679582, 1345.5241482130202)","(32327.35745493077, 32654.914800550625)","(14.41146859774188, 14.585409051257594)","(24.024270856092397, 24.048182499757466)"
9,BW3D_YSI_20230818,1960.494902,43523.819608,12.974280,20.804204,"(1916.3731509104089, 2004.6166534459787)","(43459.7697308962, 43587.86948609012)","(12.72528031482755, 12.984920752682488)","(20.798311120976447, 20.84717915981917)"


3. Save the `csv`

In [6]:
# dgh_df.to_csv(f'{root}/data/dgh_fwl_estimation.csv', index=False)

---

### **Method 4:** Land Resource Survey Threshold (LRST)

In [7]:
def lrst_fwl_estimation(path: str,
                        vp_column: str,
                        sec_column: str,
                        threshold: float = 1452.9) -> pd.DataFrame:
    """
    Identifica la posición vertical máxima para cada pozo donde la conductividad específica
    corregida es menor o igual al umbral dado (por defecto 1452.9 uS/cm).
    
    Parameters:
        path (str): Ruta del directorio que contiene los archivos CSV de los pozos.
        vp_column (str): Nombre de la columna que contiene los valores de posición vertical.
        sec_column (str): Nombre de la columna que contiene los valores de conductividad.
        threshold (float): Umbral de conductividad para determinar la posición vertical máxima.
        
    Returns:
        pd.DataFrame: DataFrame con las posiciones verticales máximas para cada pozo.
    """
    results = []
    
    # Validar si la ruta proporcionada existe
    if not os.path.exists(path):
        raise FileNotFoundError(f"La ruta proporcionada '{path}' no existe.")
    
    # Encontrar todos los archivos CSV en el directorio
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    if not csv_files:
        raise ValueError(f"No se encontraron archivos CSV en el directorio '{path}'.")
    
    for file in csv_files:
        try:
            df = pd.read_csv(file)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")
            continue
        
        # Verificar si las columnas requeridas están presentes
        if vp_column not in df.columns or sec_column not in df.columns:
            print(f"Columnas requeridas no encontradas en {file}")
            continue
        
        # Filtrar y eliminar valores nulos para las columnas requeridas
        df_filtered = df[[vp_column, sec_column]].dropna()
        
        # Filtrar datos donde la conductividad es menor o igual al umbral
        df_low_cond = df_filtered[df_filtered[sec_column] <= threshold]
        
        # Si hay datos que cumplen con el criterio, encontrar la posición vertical máxima
        if not df_low_cond.empty:
            max_vp = df_low_cond[vp_column].max()
        else:
            max_vp = None
        
        # Extraer el ID del pozo del nombre del archivo (nombre base sin extensión)
        well_id = os.path.splitext(os.path.basename(file))[0]
        
        # Si el ID termina con "_rowdy", eliminar ese sufijo
        if well_id.endswith('_rowdy'):
            well_id = well_id[:-6]
        
        # Agregar los resultados para el pozo actual
        results.append({
            "ID": well_id,
            "vp_fwl_lrst": max_vp
        })
    
    summary_df = pd.DataFrame(results)
    return summary_df


In [8]:
lrst_df = lrst_df = lrst_fwl_estimation(
    path=f'{root}/data/rawdy',
    vp_column='Vertical Position [m]',
    sec_column='Corrected sp Cond [uS/cm]',
    threshold=1452.9
)

lrst_df

,ID,vp_fwl_lrst
0,AW1D_YSI_20230826,6.120
1,AW2D_YSI_20230815,13.103
2,AW5D_YSI_20230824,12.580
3,AW6D_YSI_20230815,13.016
4,AW7D_YSI_20230814,12.881
5,BW10D_YSI_20230825,9.710
6,BW11D_YSI_20230823,12.929
7,BW1D_YSI_20230824,NaN
8,BW2D_YSI_20230819,14.038
9,BW3D_YSI_20230818,10.784


---

### Combine with the rest of the FWL estimates.

In [9]:
# Merge with dgh_df first
df_merged = df_estimations.merge(dgh_df[['ID', 'breakpoint_1 (vp)']], on="ID", how="left")
df_merged["vp_fwl_dgh"] = df_merged["breakpoint_1 (vp)"]
df_merged.drop(columns=["breakpoint_1 (vp)"], inplace=True)

# Now merge with lrst_df to add vp_fwl_lrst
df_merged = df_merged.merge(lrst_df[['ID', 'vp_fwl_lrst']], on="ID", how="left")

df_merged

,ID,total_bp_ic,fwl_bp_ic,vp_fwl_ic,total_bp_bic,fwl_bp_bic,vp_fwl_bic,vp_fwl_dgh,COMENTARIOS,SEC Profile Features,vp_fwl_lrst_x,vp_fwl_lrst_y,vp_fwl_lrst
0,AW1D_YSI_20230826,6,2,16.317073,3,1,17.169116,16.386594,NaN,NaN,6.120,6.120,6.120
1,AW2D_YSI_20230815,7,2,9.563224,3,1,9.907636,11.319882,NaN,NaN,13.103,13.103,13.103
2,AW5D_YSI_20230824,5,1,14.134225,4,1,13.652858,12.090368,NaN,NaN,12.580,12.580,12.580
3,AW6D_YSI_20230815,5,2,13.216555,2,1,13.561559,13.844801,NaN,NaN,13.016,13.016,13.016
4,AW7D_YSI_20230814,2,1,13.290227,2,1,13.290227,13.316418,No está correctamente cortado el perfil,NaN,12.881,12.881,12.881
5,BW1D_YSI_20230824,3,1,14.125652,3,1,14.125652,14.117028,NaN,NaN,NaN,NaN,NaN
6,BW2D_YSI_20230819,3,1,13.997452,3,1,13.997452,14.581115,NaN,NaN,14.038,14.038,14.038
7,BW3D_YSI_20230818,4,1,11.701564,4,1,11.701564,12.974280,NaN,NaN,10.784,10.784,10.784
8,BW4D_YSI_20230816,2,1,14.358411,2,1,14.358411,14.124974,NaN,NaN,14.014,14.014,14.014
9,BW5D_YSI_20230822,4,2,12.393402,2,1,11.527182,10.262300,NaN,NaN,NaN,NaN,NaN


In [11]:
# Eliminar las columnas vp_fwl_lrst_x y vp_fwl_lrst_y si existen
if 'vp_fwl_lrst_x' in df_merged.columns:
    df_merged.drop(columns=['vp_fwl_lrst_x'], inplace=True)
    
if 'vp_fwl_lrst_y' in df_merged.columns:
    df_merged.drop(columns=['vp_fwl_lrst_y'], inplace=True)
    
# Verificar las columnas después de eliminar
print("Columnas después de eliminar vp_fwl_lrst_x y vp_fwl_lrst_y:")
print(df_merged.columns.tolist())


Columnas después de eliminar vp_fwl_lrst_x y vp_fwl_lrst_y:
['ID', 'total_bp_ic', 'fwl_bp_ic', 'vp_fwl_ic', 'total_bp_bic', 'fwl_bp_bic', 'vp_fwl_bic', 'vp_fwl_dgh', 'COMENTARIOS', 'SEC Profile Features', 'vp_fwl_lrst']


1. Save the `csv`

In [12]:
df_merged.to_csv(f'{root}/data/selection_fwl_ic_bic_dgh.csv', 
                index=False
                )